In [9]:
import pandas as pd
import numpy as np
import requests
import json
import time
# Parameters
startDate = '2022-01-01'
endDate = '2022-01-17'

rivers_reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

sites = rivers_reference.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)

meta = pd.DataFrame()
df = pd.DataFrame()
for i in range(1):# range(len(rivers_reference)):
    lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
    long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
    baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
    # Search API for Lat Long Position
    r = requests.get('https://api.weather.gov/points/'+lat+','+long)
    json_data = r.json()

    # Find Stations in area
    url = json_data['properties']['observationStations']

    # Select first Station for testing
    station_list = requests.get(url).json()['observationStations']
    station = station_list[0]
    print(rivers_reference['USGS Name'][i])
    print('number of stations: ',len(station_list))

    # Retrieve Forecast for Lat Long Position
    forecast_json = requests.get(json_data['properties']['forecast']).json()
    try: 
        if forecast_json['status'] == 500:
            continue
    except:
        KeyError
    forecast = forecast_json['properties']['periods']
    Forecast_df = pd.DataFrame(forecast).drop(columns='number')
    Forecast_df['Lat'] = lat
    Forecast_df['Long'] = long
    Forecast_df['Station'] = baseStation

RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
number of stations:  77


In [11]:
Forecast_df.columns

Index(['name', 'startTime', 'endTime', 'isDaytime', 'temperature',
       'temperatureUnit', 'temperatureTrend', 'windSpeed', 'windDirection',
       'icon', 'shortForecast', 'detailedForecast', 'Lat', 'Long', 'Station'],
      dtype='object')

In [13]:
import pandas as pd
import numpy as np
import requests
import json
import time
# Parameters
startDate = '2022-01-01'
endDate = '2022-01-17'

rivers_reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

sites = rivers_reference.USGS_ID.tolist()
string = ''

for i in sites:
    string = string+','+str(i)
sites=string[1:]

# URL to load json of data 
# Use format=rmd for readable URL
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)
link = ('https://waterservices.usgs.gov/nwis/iv/?format=json&sites='+sites)
r=requests.get(link)
Geodata = json.loads(r.text)

meta = pd.DataFrame()
df = pd.DataFrame()
for i in range(len(rivers_reference)):
    lat = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['latitude'])
    long = str(Geodata['value']['timeSeries'][i]['sourceInfo']['geoLocation']['geogLocation']['longitude'])
    baseStation = Geodata['value']['timeSeries'][i]['sourceInfo']['siteName']
    # Search API for Lat Long Position
    r = requests.get('https://api.weather.gov/points/'+lat+','+long)
    json_data = r.json()

    # Find Stations in area
    url = json_data['properties']['observationStations']

    # Select first Station for testing
    station_list = requests.get(url).json()['observationStations']
    station = station_list[0]
    print(rivers_reference['USGS Name'][i])
    print('number of stations: ',len(station_list))

    # Retrieve Forecast for Lat Long Position
    forecast_json = requests.get(json_data['properties']['forecast']).json()
    try: 
        if forecast_json['status'] == 500:
            continue
    except:
        KeyError
    forecast = forecast_json['properties']['periods']
    Forecast_df = pd.DataFrame(forecast).drop(columns='number')
    Forecast_df['Lat'] = lat
    Forecast_df['Long'] = long
    Forecast_df['Station'] = baseStation

    full_Forecast = pd.read_csv('Gov_Weather_Forecasts.csv')
    full_Forecast = full_Forecast.append(Forecast_df[['name', 'startTime', 'endTime', 'isDaytime', 'temperature',
       'temperatureUnit', 'temperatureTrend', 'windSpeed', 'windDirection',
       'icon', 'shortForecast', 'detailedForecast', 'Lat', 'Long', 'Station']])
       
    full_Forecast.drop_duplicates(inplace=True)
    full_Forecast.to_csv('Gov_Weather_Forecasts.csv')
    
    print('number of forecasts: ',len(forecast))
    print('')
    # Locate first Station during timeframe

    observationURL = station+'/observations?start='+startDate+\
        'T00%3A00%3A00-00%3A00&end='+endDate+'T00%3A00%3A00-00%3A00'

    payload_meta = pd.DataFrame(requests.get(observationURL).json()['features'])
    if payload_meta.shape[0] > 0:
        payload_meta['longitude'] = payload_meta.geometry[0]['coordinates'][0]
        payload_meta['latitude'] = payload_meta.geometry[0]['coordinates'][1]
        payload_meta['base_latitude'] = lat
        payload_meta['base_longitude'] = long
        payload_meta['USGS Name'] = rivers_reference['USGS Name'][i]
        payload = pd.DataFrame(payload_meta.properties.to_dict()).transpose()
        payload_meta.drop(columns=['type','properties','geometry'],inplace=True)
        meta = meta.append(payload_meta)
        
        payload['elevation(m)'] = payload['elevation'].apply(lambda x: x['value'])
        payload['temperature(c)'] = payload['temperature'].apply(lambda x: x['value'])
        payload['dewpoint(c)'] = payload['dewpoint'].apply(lambda x: x['value'])
        payload['windDirection(angle)'] = payload['windDirection'].apply(lambda x: x['value'])
        payload['windSpeed(km/hr)'] = payload['windSpeed'].apply(lambda x: x['value'])
        payload['barometricPressure(Pa)'] = payload['barometricPressure'].apply(lambda x: x['value'])
        payload['relativeHumidity(%)'] = payload['relativeHumidity'].apply(lambda x: x['value'])
        payload.drop(columns=['elevation','icon', 'presentWeather', 'temperature', 'dewpoint',
            'windDirection', 'windSpeed', 'windGust', 'barometricPressure',
            'seaLevelPressure', 'visibility', 'maxTemperatureLast24Hours',
            'minTemperatureLast24Hours',
            'relativeHumidity', 'windChill', 'heatIndex', 'cloudLayers'],inplace=True)
        df = df.append(payload)
    time.sleep(2)

full_df = pd.read_csv('Gov_Weather.csv')
full_df = full_df.append(df)
full_df = full_df[['@id','@type','station','timestamp','rawMessage','textDescription','precipitationLastHour','precipitationLast3Hours','precipitationLast6Hours'
                    ,'elevation(m)','temperature(c)','dewpoint(c)','windDirection(angle)','windSpeed(km/hr)','barometricPressure(Pa)','relativeHumidity(%)']]

full_df.drop_duplicates(inplace=True)

full_df[['@id','@type','station','timestamp','rawMessage','textDescription','precipitationLastHour','precipitationLast3Hours','precipitationLast6Hours'
                    ,'elevation(m)','temperature(c)','dewpoint(c)','windDirection(angle)','windSpeed(km/hr)','barometricPressure(Pa)','relativeHumidity(%)']].to_csv('Gov_Weather.csv',index=False)

full_meta = pd.read_csv('Meta_Gov_Weather.csv')
full_meta = full_meta.append(meta)
full_meta = full_meta[['id','longitude','latitude','base_latitude','base_longitude','USGS Name']]

full_meta.drop_duplicates(inplace=True)

full_meta[['id','longitude','latitude','base_latitude','base_longitude','USGS Name']].to_csv('Meta_Gov_Weather.csv',index=False)

RUSSIAN R A HACIENDA BRIDGE NR GUERNEVILLE CA
number of stations:  77
number of forecasts:  14

NAVARRO R NR NAVARRO CA
number of stations:  77
number of forecasts:  14

NOYO R NR FORT BRAGG CA
number of stations:  77
number of forecasts:  14

SAN LORENZO R A BIG TREES CA
number of stations:  77
number of forecasts:  14

MAD R NR ARCATA CA
number of stations:  77
number of forecasts:  14

PUTAH C NR WINTERS CA
number of stations:  77
number of forecasts:  14

TRUCKEE R A FARAD CA
number of stations:  77
number of forecasts:  14

LITTLE TRUCKEE R AB BOCA RES NR TRUCKEE CA
number of stations:  77
number of forecasts:  14

SMITH R NR CRESCENT CITY CA
number of stations:  77
number of forecasts:  14

1
number of stations:  41
number of forecasts:  14

EEL R A FORT SEWARD CA
number of stations:  41
number of forecasts:  14

EEL R A SCOTIA CA
number of stations:  41
number of forecasts:  14

2
number of stations:  41
number of forecasts:  14

SAN LORENZO R A SANTA CRUZ CA
number of stations:

TypeError: unhashable type: 'dict'